In [1]:
%load_ext autoreload
%autoreload 2
%cd /disks/sdb/zhouyi/frequency
import os
os.environ['TORCH_HOME'] = "/disks/sdb/torch_home"

from awesome_glue.task import *
from awesome_glue.config import Config
from collections import defaultdict
from tabulate import tabulate
from allennlp.data import Instance


/disks/sdb/zhouyi/frequency


I0403 04:02:58.878138 140607966021440 file_utils.py:38] PyTorch version 1.4.0 available.
Loading faiss with AVX2 support.


In [23]:
torch.cuda.set_device(2)

config = Config()

task = Task(config)
task.from_pretrained()
config

cache for IMDB-spacy.data exists
*** load IMDB-spacy.data from cache cost 34.3 seconds
cache for IMDB-counter.vec exists
*** load IMDB-counter.vec from cache cost 0.0922 seconds


I0403 04:22:40.062784 140607966021440 predictor.py:157] Identity transformation detected.


cache for IMDB-glove.vec exists
*** load IMDB-glove.vec from cache cost 0.0935 seconds
Load model from saved/models/IMDB-dlstm-5.0/best.th


Basic Args:
	--task_id=IMDB
	--embed=d
	--arch=lstm
	--pretrain=glove
	--_model_name=
	--mode=attack
	--dir_temp=5.0
	--gnn_type=mean
	--gnn_hop=1
	--aug_data=
	--adv_iter=0
	--adv_policy=hot
	--adv_replace_num=5
	--adv_constraint=True
	--pred_ensemble=1
	--pred_transform=
	--pred_transform_args=
	--attack_method=pwws
	--attack_vectors=counter
	--attack_data_split=dev
	--attack_size=200
	--attack_gen_adv=False
	--adv_data=nogit/AGNEWS-lstm.hotflip.adv.tsv
	--alchemist=False
	--seed=2
	--cuda=3
Deduced Args:
	--model_name=IMDB-dlstm-5.0
	--tokenizer=spacy

In [26]:
inst = task.train_data[130]
print(len(inst['sent'].tokens))


# torch.cuda.empty_cache()
# r = task.predictor._model.forward_on_instances([inst for _ in range(400)])
# torch.cuda.empty_cache()
r = task.predictor.predict_batch_json([allenutil.as_json(inst) for _ in range(4000)])
len(r)

161
SPLIT
SPLIT
SPLIT
SPLIT
SPLIT
SPLIT


4000

In [ ]:
data = load_data("IMDB", "bert")

cache for IMDB-bert.data exists


In [213]:
train_data = data['data'][0]
sampler = BucketBatchSampler(
    data_source=train_data,
    batch_size=16,
)
for batch in iter(sampler):
    print(len(train_data[batch[-1]]['sent'].tokens))

I0402 10:35:39.883920 140358541928256 bucket_batch_sampler.py:83] No sorting keys given; trying to guess a good one
I0402 10:35:39.906545 140358541928256 bucket_batch_sampler.py:85] Using ['sent'] as the sorting keys


20
21
18
19
22
20
20
19
21
21
21
22
20
23
23
22
24
22
21
22
22
22
21
23
22
22
24
24
24
23
24
24
25
24
23
24
24
25
24
24
26
26
25
25
22
26
24
24
24
23
22
24
26
26
25
24
26
25
26
24
25
23
25
23
23
25
24
27
27
24
24
26
27
23
27
26
27
26
27
27
26
24
26
26
27
27
27
26
27
28
27
26
26
28
28
28
26
26
28
28
28
25
27
26
26
28
24
28
25
25
28
26
26
28
25
27
29
29
29
28
26
29
25
29
27
29
26
29
28
28
27
25
26
26
28
28
25
28
27
27
28
26
29
26
29
29
27
27
29
29
27
28
25
29
27
26
26
30
28
29
27
28
25
26
29
29
27
25
28
28
29
27
28
29
26
26
27
28
27
30
30
28
29
30
28
29
26
28
26
26
29
29
28
31
26
31
31
28
28
28
27
27
28
31
31
30
27
31
30
27
31
30
30
27
30
30
28
29
29
27
29
28
28
28
28
28
31
27
29
31
27
29
30
28
31
30
28
31
27
30
31
30
31
31
31
32
27
29
32
32
28
30
30
27
27
29
27
32
29
32
27
31
29
28
30
30
27
29
29
28
28
31
28
32
32
27
28
28
30
32
28
30
29
31
30
30
29
27
29
28
32
33
30
31
31
33
32
32
28
29
33
28
32
30
32
29
32
31
29
32
32
32
32
29
32
29
30
32
31
33
29
29
32
33
32
28
32
32
32
31
28
28
33
3

In [13]:
embed_searcher = EmbeddingSearcher(weight, data['vocab'].get_token_index, data['vocab'].get_token_from_index)

In [15]:
embed_searcher.show_embedding_info('cos')

*** Statistics of parameters and 2-norm ***
[INFO] Param Mean -0.0006 Std 0.0539 Max 0.5341
[INFO] Norm Mean 0.8714 Std 0.3348 Max 1.0000
*** Statistics of distances in a N-nearest neighbourhood ***
    N    mean    std
-----  ------  -----
    5    0.32   0.18
   10    0.39   0.18
   20    0.46   0.17
   50    0.55   0.16
  100    0.62   0.14
  200    0.68   0.12
  500    0.74   0.10
10000    0.94   0.07
20000    0.97   0.08


In [17]:
vocab = Vocabulary(padding_token='[PAD]', oov_token='[UNK]')
from allennlp.data.token_indexers import PretrainedTransformerIndexer
PretrainedTransformerIndexer("bert-base-uncased", "tokens")._add_encoding_to_vocabulary_if_needed(vocab)

In [21]:
vocab.get_vocab_size("tokens")

30522

In [10]:
from allennlp.training.checkpointer import Checkpointer
ckpter = Checkpointer('../saved/models/AGNEWS-lstm')
ckpter.find_latest_checkpoint()[0]

'../saved/models/AGNEWS-lstm/model_state_epoch_5.th'

In [24]:
w = _read_pretrained_embeddings_file(
            WORD2VECS['counter'],
            embedding_dim=EMBED_DIM['counter'],
            vocab=task.vocab,
            namespace="tags")

I0401 07:42:19.564311 140387497060160 embedding.py:383] Reading pretrained embeddings from file


I0401 07:42:21.463958 140387497060160 embedding.py:421] Initializing pre-trained embedding layer


I0401 07:42:21.773792 140387497060160 embedding.py:439] Pretrained embeddings were found for 20994 out of 30522 tokens


In [29]:
w[:, 0].nonzero().size()

torch.Size([20994, 1])

In [9]:
task.model.bert_model.transformer_model.embeddings.word_embeddings

Embedding(30522, 768, padding_idx=0)

In [44]:
task.model.bert_model.embeddings.word_embeddings

True

In [14]:
from allennlp.data import Instance
from allennlp.data.fields import TextField
from allennlpx.allenutil import as_sentence
allenutil.as_sentence(task.train_data[0].fields['sent1'])
def as_json(instance: Instance):
    ret = {}
    for k, v in instance.items():
        if isinstance(v, TextField):
            ret[k] = as_sentence(v)
    return ret
as_json(task.train_data[0])

{'sent1': 'A person on a horse jumps over a broken down airplane .',
 'sent2': 'A person is outdoors , on a horse .'}

In [3]:
forbidden_words = DEFAULT_IGNORE_TOKENS
forbidden_words.extend([
    line.rstrip('\n') for line in open(TASK_SPECS['SST']['banned_words'])
])
# forbidden_words += stopwords.words("english")
high_words = FreqUtil.topk_frequency(task.vocab, 50, 'most', forbidden_words)
high_words

['the',
 'a',
 'and',
 'of',
 'to',
 "'s",
 'is',
 'that',
 'in',
 'it',
 'as',
 'with',
 'an',
 'film',
 'for',
 'its',
 'movie',
 'this',
 '`',
 'you',
 'be',
 'but',
 'on',
 'by',
 'more',
 'one',
 '--',
 'at',
 'than',
 'has',
 'from',
 'about',
 'his',
 'are',
 'so',
 'all',
 'or',
 'have',
 'most',
 'out',
 'story',
 'too',
 'into',
 'up',
 'who',
 'characters',
 'i',
 'comedy',
 'if',
 'just']

In [4]:
config = Config()
config.task_id = 'SST'
task = Task(config)
task.from_pretrained()
config


# values = list(task.vocab.get_index_to_token_vocabulary().values())
values = high_words
results = defaultdict(lambda: [], {})
from allennlp.common.util import lazy_groups_of
for group in lazy_groups_of(values, 1024):
    result = task.predictor.predict_batch_json([{"sent": ele} for ele in group])
    for i in range(len(result[0]['probs'])):
        results[i].extend([ele['probs'][i] for ele in result])
pairs = defaultdict(lambda: [], {})
for i in range(len(results)):
    pairs[i] = sorted(zip(values, results[i]), key=lambda x: x[1], reverse=True)

cache for SST-spacy.data exists
*** load SST-spacy.data from cache cost 4.23 seconds
cache for SST-glove.vec exists
*** load SST-glove.vec from cache cost 0.0137 seconds
Load model from saved/models/SST-lstm/best.th


In [5]:
config._model_name = 'SST-glstm'
task = Task(config)
task.from_pretrained()
config

# values = list(task.vocab.get_index_to_token_vocabulary().values())
values = high_words
results = defaultdict(lambda: [], {})
from allennlp.common.util import lazy_groups_of
for group in lazy_groups_of(values, 1024):
    result = task.predictor.predict_batch_json([{"sent": ele} for ele in group])
    for i in range(len(result[0]['probs'])):
        results[i].extend([ele['probs'][i] for ele in result])
pairs2 = defaultdict(lambda: [], {})
for i in range(len(results)):
    pairs2[i] = sorted(zip(values, results[i]), key=lambda x: x[1], reverse=True)

cache for SST-spacy.data exists
*** load SST-spacy.data from cache cost 3.22 seconds
cache for SST-glove.vec exists
*** load SST-glove.vec from cache cost 0.0091 seconds
Load model from saved/models/SST-glstm/best.th


In [32]:
from copy import deepcopy
x=[]
xs = [x] * 2
ys = [deepcopy(ele) for ele in xs]
ys[0].append(1)
print(ys)
ys = deepcopy(xs)
ys[0].append(1)
print(ys)

[[1], []]
[[1], [1]]


In [44]:
from goto import goto, label
for i in range(9):
    for j in range(9):
        for k in range(9):
            print("I am trapped, please rescue!")
            if k == 2:
                goto .breakout # breaking out from a deeply nested loop
label .breakout
print("Freedom!")

ModuleNotFoundError: No module named 'goto'

In [6]:
for i in range(len(pairs)):
    tab = []
    for ele1, score1 in pairs[i]:
        for ele2, score2 in pairs2[i]:
            if ele1 == ele2:
                tab.append((ele1, round(score1, 2),  round(score2, 2)))
    print(tabulate(tab, headers=['word', 'normal', 'adv']))

word          normal    adv
----------  --------  -----
too             0.96   0.95
or              0.82   0.98
if              0.76   0.86
from            0.71   0.44
than            0.66   0.98
have            0.6    0.03
out             0.58   0.9
its             0.57   0.42
on              0.56   0.38
just            0.56   0.91
`               0.54   0.4
at              0.53   0.44
as              0.53   0.32
but             0.53   0.35
by              0.51   0.46
about           0.48   0.31
of              0.47   0.49
movie           0.47   0.38
one             0.47   0.41
that            0.47   0.48
for             0.47   0.3
a               0.46   0.98
so              0.45   0.13
be              0.45   0.3
most            0.45   0.91
the             0.44   0.47
more            0.44   0.02
into            0.43   0.44
with            0.42   0.45
in              0.42   0.39
his             0.42   0.16
it              0.42   0.5
all             0.41   0.17
story           0.4    0.